In [19]:
pip install customtkinter opencv-python Pillow pyttsx3 ultralytics


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
import os
import pyttsx3
import cv2
import numpy as np
import customtkinter as ctk
from tkinter import filedialog, Canvas
from PIL import Image, ImageTk
from customtkinter import CTkImage
from ultralytics import YOLO

# Initialize model and class labels
model = YOLO(r"C:\Users\gopik\Downloads\best (5).pt")

class_labels = {
    0: "AnnualCrop", 1: "Forest", 2: "HerbaceousVegetation", 3: "Highway", 4: "Industrial",
    5: "Pasture", 6: "PermanentCrop", 7: "Residential", 8: "River", 9: "SeaLake"
}

engine = pyttsx3.init()
alert_terrain = ""
history = []
video_cap = None
video_running = False
frame_resized = None
terrain_classes = list(class_labels.values())
#background_image_path = r"C:\Users\gopik\OneDrive\Pictures\bgdrone.jpg"
#image_path = r"C:\Users\gopik\OneDrive\Pictures\user.png"
y_position = 10 

def show_signin_window():
    ctk.set_appearance_mode("dark")
    ctk.set_default_color_theme("blue")

    signin_root = ctk.CTk()
    signin_root.title("Sign In")
    signin_root.geometry("1200x700")

    # Set the background image (cover the whole window)
    background_image_path = r"C:\Users\gopik\OneDrive\Pictures\blue.jpg"  # replace with your background image path
    if os.path.exists(background_image_path):
        bg_img = Image.open(background_image_path)
        bg_img = bg_img.resize((1200, 700))  # Resize to window size
        bg_photo = ImageTk.PhotoImage(bg_img)

        bg_label = ctk.CTkLabel(signin_root, image=bg_photo)
        bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # Cover the entire window

    frame = ctk.CTkFrame(signin_root, corner_radius=15, fg_color="black", width=400, height=400)
    frame.place(relx=0.5, rely=0.5, anchor="center")

    # Logo
    image_path = r"C:\Users\gopik\OneDrive\Pictures\user.png"
    if os.path.exists(image_path):
        img = Image.open(image_path).resize((100, 100))
        ctk_img = CTkImage(light_image=img, dark_image=img, size=(100, 100))
        img_label = ctk.CTkLabel(frame, image=ctk_img, text="")
        img_label.pack(pady=(20, 10))

    # Username
    username_label = ctk.CTkLabel(frame, text="Username", font=ctk.CTkFont(size=16))
    username_label.pack(pady=(10, 0))
    username_entry = ctk.CTkEntry(frame, width=250, font=ctk.CTkFont(size=14))
    username_entry.pack(pady=(5, 10))

    # Password
    password_label = ctk.CTkLabel(frame, text="Password", font=ctk.CTkFont(size=16))
    password_label.pack(pady=(10, 0))
    password_entry = ctk.CTkEntry(frame, width=250, font=ctk.CTkFont(size=14), show="*")
    password_entry.pack(pady=(5, 10))

    # Error label
    error_label = ctk.CTkLabel(frame, text="", text_color="red", font=ctk.CTkFont(size=14))
    error_label.pack(pady=(5, 0))
    
    

    def authenticate():
        username = username_entry.get()
        password = password_entry.get()

        # Hardcoded credentials for now
        if username == "admin" and password == "1234":
            signin_root.destroy()
            show_intro_window()
        else:
            error_label.configure(text="Invalid username or password!")

    # Sign In button
    signin_button = ctk.CTkButton(frame, text="Sign In", command=authenticate, font=ctk.CTkFont(size=16))
    signin_button.pack(pady=(20, 10))

    signin_root.mainloop()



def show_intro_window():
    ctk.set_appearance_mode("dark")
    ctk.set_default_color_theme("blue")

    intro_root = ctk.CTk()
    intro_root.title("Terrain Alert Setup")
    intro_root.geometry("1200x700")

    image_path = r"C:\Users\gopik\Downloads\—Pngtree—aerial view of a white_15892336.png"  # Replace with your actual image path
    img = Image.open(image_path)
    img = img.resize((200, 200))  # Resize as needed
    ctk_img = CTkImage(light_image=img, dark_image=img, size=(200, 200))

    image_label = ctk.CTkLabel(intro_root, image=ctk_img, text="")
    image_label.pack(pady=(30, 10))

    label = ctk.CTkLabel(intro_root, text="Enter Alert Terrain", font=ctk.CTkFont(size=20, weight="bold"))
    label.pack(pady=20)

    entry = ctk.CTkEntry(intro_root, font=ctk.CTkFont(size=16), width=250)
    entry.pack(pady=10)

    def proceed_to_main():
        global alert_terrain
        alert_terrain = entry.get()
        intro_root.destroy()
        show_main_window()

    next_button = ctk.CTkButton(intro_root, text="Next", command=proceed_to_main, font=ctk.CTkFont(size=16))
    next_button.pack(pady=20)

    intro_root.mainloop()

def speak_terrain(terrain_name):
    engine.say(f"Alert {terrain_name} Detected !")
    engine.runAndWait()

def get_terrain_confidence(img_path):
    results = model(img_path)
    if results and len(results[0].boxes.cls) > 0:
        cls_id = int(results[0].boxes.cls[0])
        conf = float(results[0].boxes.conf[0]) * 100
        return class_labels.get(cls_id, "Unknown"), round(conf, 2)
    return "Unknown", 0.0

def show_main_window():
    global alert_terrain, video_cap, video_running, frame_resized

    root = ctk.CTk()
    root.geometry("1200x700")
    root.title("Terrain Detection System")
    root.configure(bg="black")

    root.grid_rowconfigure((0, 1, 2, 3, 4), weight=1)
    root.grid_columnconfigure((0, 1, 2), weight=1)


    label = ctk.CTkLabel(root, text="Spotting the Terrain", font=ctk.CTkFont(size=30, weight="bold"), text_color="white")
    label.grid(row=0, column=0, columnspan=3, pady=(20, 10), sticky="n")
    

    # Flowchart frame (right side)
    flowchart_frame = ctk.CTkFrame(root, width=300, height=800, corner_radius=10)
    flowchart_frame.grid(row=1, column=2, rowspan=3, padx=(10, 20), pady=10, sticky="nsew")


    # Canvas for displaying image or video
    global media_canvas
    media_canvas = Canvas(root, width=800, height=600, bg="black", highlightthickness=0)
    media_canvas.grid(row=1, column=0, columnspan=2, padx=(50, 10), pady=10, sticky="n")


    # Flowchart canvas with scroll
    flowchart_canvas = Canvas(flowchart_frame, bg="black", highlightthickness=0, width=350, height=800)
    flowchart_canvas.pack(fill="both", expand=True)

    def on_mousewheel(event):
        flowchart_canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

    flowchart_canvas.bind_all("<MouseWheel>", on_mousewheel)
    flowchart_canvas.bind_all("<Button-4>", lambda e: flowchart_canvas.yview_scroll(-1, "units"))
    flowchart_canvas.bind_all("<Button-5>", lambda e: flowchart_canvas.yview_scroll(1, "units"))
     # Global variable to track the vertical position of the buttons

    def draw_flowchart(predicted_terrain, selected_terrain):
        global y_position  # Access the global y_position variable
        
        # Display only the most recent terrain
        is_match = predicted_terrain.lower() == selected_terrain.lower()
    
        # Create a button to represent the predicted terrain
        btn = ctk.CTkButton(
            master=flowchart_canvas,
            text=selected_terrain,  
            width=240,
            height=40,
            state="disabled",
            fg_color="red" if is_match else "gray25",
            text_color="white",
            font=ctk.CTkFont(size=14),
            corner_radius=10
        )
    
        # Place the button at the current y_position
        flowchart_canvas.create_window(20, y_position, anchor="nw", window=btn)
        spacing = 60
        # Update y_position for the next button
        y_position += spacing   # Adjust the spacing between buttons as needed
    
        # Adjust the flowchart canvas to fit the newly drawn terrain (set the scroll region)
        flowchart_canvas.config(scrollregion=flowchart_canvas.bbox("all"))



    def display_frame():
        global video_cap, video_running, frame_resized
        if video_cap and video_running:
            ret, frame = video_cap.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_resized = cv2.resize(frame_rgb, (800, 600))
                results = model(frame_resized)

                for box in results[0].boxes:
                    cls_id = int(box.cls[0])
                    conf = float(box.conf[0])
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{class_labels.get(cls_id, 'Unknown')} {conf:.2f}"
                    cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(frame_resized, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

                img = ImageTk.PhotoImage(Image.fromarray(frame_resized))
                media_canvas.create_image(0, 0, anchor="nw", image=img)
                media_canvas.image = img
                root.after(30, display_frame)
            else:
                video_cap.release()
                video_running = False

    def upload_and_display():
        global video_cap, video_running, frame_resized
        filepath = filedialog.askopenfilename(filetypes=[("Media files", "*.jpg *.jpeg *.png *.mp4 *.avi *.mov *.mkv")])
        if not filepath:
            return

        ext = os.path.splitext(filepath)[1].lower()

        if video_cap:
            video_running = False
            video_cap.release()

        if ext in ['.jpg', '.jpeg', '.png']:
            img = cv2.imread(filepath)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            frame_resized = cv2.resize(img_rgb, (800, 600))
            results = model(frame_resized)

            highest_confidence = -1
            highest_confidence_box = None
            for box in results[0].boxes:
                conf = float(box.conf[0])
                if conf > highest_confidence:
                    highest_confidence = conf
                    highest_confidence_box = box

            if highest_confidence_box:
                cls_id = int(highest_confidence_box.cls[0])
                x1, y1, x2, y2 = map(int, highest_confidence_box.xyxy[0])
                label = f"{class_labels.get(cls_id, 'Unknown')} {highest_confidence:.2f}"
                cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame_resized, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

            img_pil = Image.fromarray(frame_resized)
            img_tk = ImageTk.PhotoImage(img_pil)
            media_canvas.create_image(0, 0, anchor="nw", image=img_tk)
            media_canvas.image = img_tk

            terrain, confidence = get_terrain_confidence(filepath)
            terrain_info_label.configure(text=f"Detected Terrain: {terrain}\nConfidence: {confidence}%")
            
            history.append(terrain)
            draw_flowchart(alert_terrain, terrain) 

            if terrain.lower() == alert_terrain.lower():
                speak_terrain(terrain)

        elif ext in ['.mp4', '.avi', '.mov', '.mkv']:
            video_cap = cv2.VideoCapture(filepath)
            video_running = True
            display_frame()

    def save_file():
        global frame_resized
        filepath = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("Image Files", "*.jpg *.png *.jpeg")])
        if filepath and frame_resized is not None:
            img = Image.fromarray(frame_resized)
            img.save(filepath)

    # Buttons
    save_button = ctk.CTkButton(root, text="Save", command=save_file, font=ctk.CTkFont(size=16))
    save_button.grid(row=2, column=0, padx=10, pady=5, sticky="e")

    upload_button = ctk.CTkButton(root, text="Upload", command=upload_and_display, font=ctk.CTkFont(size=16))
    upload_button.grid(row=2, column=1, padx=10, pady=5, sticky="w")

    # Info label
    global terrain_info_label
    terrain_info_label = ctk.CTkLabel(root, text="Detected Terrain: None\nConfidence: 0.0%", font=ctk.CTkFont(size=16))
    terrain_info_label.grid(row=3, column=0, columnspan=2, pady=10, sticky="n")

    root.mainloop()

show_signin_window()




0: 480x640 1 Industrial, 82.5ms
Speed: 2.9ms preprocess, 82.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\gopik\OneDrive\Desktop\Drone\images\test\AnnualCrop_7.jpg: 640x640 1 AnnualCrop, 110.0ms
Speed: 4.3ms preprocess, 110.0ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 AnnualCrop, 88.8ms
Speed: 2.5ms preprocess, 88.8ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\gopik\OneDrive\Desktop\Drone\images\test\Forest_1359.jpg: 640x640 1 Forest, 110.3ms
Speed: 4.7ms preprocess, 110.3ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 Residential, 90.4ms
Speed: 2.1ms preprocess, 90.4ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\gopik\OneDrive\Desktop\Drone\images\test\Residential_2735.jpg: 640x640 1 Residential, 98.7ms
Speed: 3.8ms preprocess, 98.7ms inference, 1.2ms postprocess per image at shape (1, 3, 